In [ ]:
!git clone https://github.com/zackproser/portfolio.git

In [33]:
!pip install langchain_community langchain_pinecone langchain_openai tqdm unstructured langchainhub langchain_text_splitters

In [35]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob
from tqdm import tqdm

In [ ]:
loader = DirectoryLoader('portfolio', glob="src/content/blog/**/*.mdx", show_progress=True, use_multithreading=True)

In [ ]:
docs = loader.load()

In [ ]:
docs

In [ ]:
from google.colab import userdata
# Set the API keys
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [36]:
# Assuming you've already imported necessary libraries and blog_posts is populated as above
index_name = 'zack-writing'
# Initialize embeddings and the vector store
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-large'
)

# Initialize the Pinecone vector store
vectorstore = PineconeVectorStore(embedding=embeddings, index_name=index_name)

# Set the batch size (adjust as needed)
batch_size = 100

# Upsert documents in batches
for i in tqdm(range(0, len(split_docs), batch_size)):
    batch = split_docs[i : i + batch_size]
    vectorstore.add_documents(batch)

100%|██████████| 22/22 [02:02<00:00,  5.55s/it]


In [ ]:
vectorstore = PineconeVectorStore(embedding=embeddings, index_name=index_name)

vectorstore.add_documents(split_docs)

In [ ]:
# Pinecone Index sanity checks
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

# Set the name of your Pinecone Index here
index_name = 'zack-portfolio'

index = pc.Index(index_name)

# This sanity check call should return stats for your Pinecone index, such as:
# {'dimension: 1536,
#  'index_fullness': 0.0,
#  'namespaces': {'', {'vector_count': 862}},
#  'total_vector_count': 862}
#
index.describe_index_stats()

In [ ]:
# Query the Pinecone index for related documents
query = "What is the programming bug?"

embeddings =  OpenAIEmbeddings()
vectorstore = PineconeVectorStore(embedding=embeddings, index_name=index_name)

vectorstore.similarity_search(query)